# Linear codes for SSS-based polynomial masking

### The SSS-based masking is a special case of GCM (Generalized Code-based masking)
### $n=3$ shares, $t=1$, $\ell=8$ bits: (3,1)-SSS (Shamir's Secret Sharing)

Parameters:

- $Z=(X + \alpha_1Y_1, X + \alpha_2Y_1, X + \alpha_3Y_1)=X\mathbf{G} + Y\mathbf{H}$ where $X, Y=(Y_1)$ and $Z$ are the sensitive variable, a mask and the protected variable, respectively, where $\alpha_i$ for $1\leq i\leq 3$ are three public points in SSS-scheme
- $\mathbf{G} = [1, 1, 1]$ and $\mathbf{H} = [\alpha_1, \alpha_2, \alpha_3]$ are two generator matrices of codes $\mathcal{C}$ and $\mathcal{D}$, resp.
- $\alpha_i\in \mathbb{F}_{2^\ell}\backslash\{0\}$ and $\alpha_1\neq\alpha_2\neq\alpha_3$, thus there are ${255}\choose{3}$ = 2731135 linear codes for (3,1)-SSS
- Each nonzero element over $\mathbb{F}_{2^\ell}$ can be denoted as $\alpha^i$ where $i\in\{0, 1, \ldots, 254\}$, the corresponding irreducible polynomial is $g(\alpha) = \alpha^8 + \alpha^4 + \alpha^3 + \alpha^2 +1$
- Due to equivalence of linear codes, we simplify the enumeration by choosing $(\alpha_1, \alpha_2, \alpha_3)=(\alpha^i, \alpha^j,\alpha^k)$ where $i=0$ and $0<j<k$. Therefore, we get 32131 linear codes

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pandas as pd # Pandas for tables
from IPython.display import Latex
from IPython.display import HTML

In [2]:
def read_log(file_name):
    
    pow_ind = []
    dim_all = []
    d_orig_w = []
    d_dual_w = []
    d_dual_b = []
    with open(file_name, 'r') as fp:
        wd = fp.read().split("\n")[:]
        len_all = 0
        for i in range(len(wd)):
            if wd[i].startswith("j, k ="):
                pow_ind.append([int(i) for i in re.findall(r"\d+", wd[i])])
                len_all = len_all + 1
            elif wd[i].startswith("Dimension:"):
                dim_all.append([int(i) for i in re.findall(r"\d+", wd[i])])
            elif wd[i].startswith("WD orig D (word):"):
                d_orig_w.append([int(i) for i in re.findall(r"\d+", wd[i])])
            elif wd[i].startswith("WD dual D (word):"):
                d_dual_w.append([int(i) for i in re.findall(r"\d+", wd[i])])
            elif wd[i].startswith("WD dual D  (bit):"):
                d_dual_b.append([int(i) for i in re.findall(r"\d+", wd[i]+wd[i+1])])
            else:
                continue
    
    return pow_ind, d_dual_b

## 1. Loading all weight enumerators

In [3]:
pow_ind, d_dual_b = read_log("./magma_paper/gen_codes_sss_3_1_8b.log") # indices and Weight distributions

print(len(pow_ind)) # 91 entries: 91 for (3,1)-SSS
#print(len(d_dual_b))

32131


### 1.1 Generating values

In [4]:
alpha_all = np.array(['$\\alpha^{%d}$'%i for i in np.arange(255)])
d_all = np.zeros(len(pow_ind))
B_all = np.zeros(len(pow_ind))
alpha_2 = np.zeros(len(pow_ind), dtype=int)
alpha_3 = np.zeros(len(pow_ind), dtype=int)
for i in range(len(pow_ind)):
    d_all[i] = d_dual_b[i][2]
    B_all[i] = d_dual_b[i][3]
    alpha_2[i] = pow_ind[i][0]
    alpha_3[i] = pow_ind[i][1]

### 1.2 Defining styles of dataframe

See more setting of dataframe from https://mode.com/example-gallery/python_dataframe_styling/

In [5]:
# Set properties for th, td and caption elements in dataframe
th_props = [('font-size', '14px'), ('text-align', 'left'), ('font-weight', 'bold'), ('background-color', '#E0E0E0')]
td_props = [('font-size', '13px'), ('text-align', 'left'), ('min-width', '80px')]
cp_props = [('font-size', '16px'), ('text-align', 'center')]
# Set table styles
styles = [dict(selector="th", props=th_props), dict(selector="td", props=td_props), dict(selector="caption", props=cp_props)]
cm_1 = sns.light_palette("red", as_cmap=True)
cm_2 = sns.light_palette("purple", as_cmap=True, reverse=True)

In [6]:
#df = pd.DataFrame({'$\\alpha_2$': alpha_all[alpha_2[:]], '$\\alpha_3$': alpha_all[alpha_3[:]], '$d_{\mathcal{D}}^\perp$': d_all[:], 
#                   '$B_{d_{\mathcal{D}}^\perp}$': B_all[:], 'Weight Enumerators': d_dual_b[:]})

pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.width', 800)

<span style="color:red"> **We omit the complete Tab. I here to avoid wasting of spaces. Hereafter, we only show all codes with maximized $d_{\mathcal{D}}^\perp=4$.** </span>

## 2. Optimal linear codes for (3,1)-SSS-based masking

### 2.1 Linear codes with $d_{\mathcal{D}}^\perp=4$

We focus on the the linear codes with greater $d_{\mathcal{D}}^\perp$, which are better in the sense of side-channel resistance (from our paper).

In [7]:
# Finding the indices of d_C=6
d_index = []
d_index_alpha_2 = []
d_index_alpha_3 = []
d_D_perp = 4
for i in range(len(d_dual_b)):
    if d_dual_b[i][2] == d_D_perp:
        d_index.append(i)
        d_index_alpha_2.append(pow_ind[i][0])
        d_index_alpha_3.append(pow_ind[i][1])

d_index = np.array(d_index)
d_index_alpha_2 = np.array(d_index_alpha_2)
d_index_alpha_3 = np.array(d_index_alpha_3)

In [8]:
print(len(d_index))
#print(d_index)

768


In [9]:
def highlight(s, threshold, column):
    is_min = pd.Series(data=False, index=s.index)
    is_min[column] = (s.loc[column] <= threshold)
    return ['background-color: gold' if is_min.any() else '' for v in is_min]

In [10]:
df_4 = pd.DataFrame({'$\\alpha_2$': np.array(alpha_all)[d_index_alpha_2], '$\\alpha_3$': np.array(alpha_all)[d_index_alpha_3], '$d_{\mathcal{D}}^\perp$': 
                     d_all[d_index], '$B_{d_{\mathcal{D}}^\perp}$': B_all[d_index], 'Weight Enumerators* (the first 22 values)': np.array(d_dual_b)[d_index]})
df_4 = df_4.sort_values(by=['$B_{d_{\mathcal{D}}^\perp}$'], ascending=True)

(df_4.style
    .apply(highlight, threshold=12, column=['$B_{d_{\mathcal{D}}^\perp}$'], axis=1)
    .background_gradient(cmap=cm_2, subset=['$B_{d_{\mathcal{D}}^\perp}$' ])
    .set_caption('Tab. II Linear codes for (3,1)-SSS-based masking with $d_{\mathcal{D}}^\perp=3$.')
    .set_table_styles(styles))

,$\alpha_2$,$\alpha_3$,$d_{\mathcal{D}}^\perp$,$B_{d_{\mathcal{D}}^\perp}$,Weight Enumerators* (the first 22 values)
707,$\alpha^{175}$,$\alpha^{247}$,4,36,"[0, 1, 4, 36, 5, 200, 6, 538, 7, 1312, 8, 2854, 9, 5088, 10, 7712, 11, 9824, 12, 10476, 175, 248]"
311,$\alpha^{72}$,$\alpha^{80}$,4,36,"[0, 1, 4, 36, 5, 200, 6, 538, 7, 1312, 8, 2854, 9, 5088, 10, 7712, 11, 9824, 12, 10476, 72, 81]"
24,$\alpha^{8}$,$\alpha^{183}$,4,36,"[0, 1, 4, 36, 5, 200, 6, 538, 7, 1312, 8, 2854, 9, 5088, 10, 7712, 11, 9824, 12, 10476, 8, 184]"
492,$\alpha^{87}$,$\alpha^{214}$,4,37,"[0, 1, 4, 37, 5, 198, 6, 537, 7, 1320, 8, 2842, 9, 5080, 10, 7748, 11, 9816, 12, 10442, 87, 215]"
485,$\alpha^{86}$,$\alpha^{214}$,4,37,"[0, 1, 4, 37, 5, 200, 6, 534, 7, 1308, 8, 2862, 9, 5108, 10, 7692, 11, 9788, 12, 10526, 86, 215]"
552,$\alpha^{127}$,$\alpha^{168}$,4,37,"[0, 1, 4, 37, 5, 198, 6, 537, 7, 1320, 8, 2842, 9, 5080, 10, 7748, 11, 9816, 12, 10442, 127, 169]"
214,$\alpha^{41}$,$\alpha^{128}$,4,37,"[0, 1, 4, 37, 5, 198, 6, 537, 7, 1320, 8, 2842, 9, 5080, 10, 7748, 11, 9816, 12, 10442, 41, 129]"
494,$\alpha^{88}$,$\alpha^{104}$,4,37,"[0, 1, 4, 37, 5, 197, 6, 542, 7, 1308, 8, 2838, 9, 5144, 10, 7700, 11, 9708, 12, 10566, 88, 105]"
213,$\alpha^{41}$,$\alpha^{127}$,4,37,"[0, 1, 4, 37, 5, 200, 6, 534, 7, 1308, 8, 2862, 9, 5108, 10, 7692, 11, 9788, 12, 10526, 41, 128]"
484,$\alpha^{86}$,$\alpha^{213}$,4,37,"[0, 1, 4, 37, 5, 200, 6, 536, 7, 1306, 8, 2848, 9, 5122, 10, 7734, 11, 9746, 12, 10456, 86, 214]"


### 2.2 Optimal codes for (3,1)-SSS-based masking

As shown in our paper, the codes satifying two conditions are optimal:

- Maximizing $d_{\mathcal{D}}^\perp$, here $\max\{d_{\mathcal{D}}^\perp\} = 4$
- Minimizing $B_{d_{\mathcal{D}}^\perp}$, here $\min\{B_{d_{\mathcal{D}}^\perp}\} = 36$

Note that we use two complementary metrics **SNR** (signal-to-noise ratio) and **MI** (mutual information) to assess the side-channel resistance of SSS-based masking with different codes.

As a result of Tab. II, we conclude that the optimal codes for IPM are genetated by $\mathbf{H}=[\alpha_1, \alpha_2,\alpha_3]$ where $(\alpha_1, \alpha_2,\alpha_3)\in\{(\alpha^0, \alpha^{175}, \alpha^{247})，(\alpha^0, \alpha^{72}, \alpha^{80})，(\alpha^0, \alpha^{8}, \alpha^{183})\}$. Note that permutation on three public points does not change the codes due to equivalence.

The generator matrices of the three optimal codes are shown below.

- $(\alpha_1, \alpha_2,\alpha_3) = (\alpha^0, \alpha^{175}, \alpha^{247})$

$$
\mathbf{H}_{optimal}=\left( \begin{matrix} \alpha^0 & \alpha^{175} & \alpha^{247} \end{matrix} \right) \in \mathbb{F}_{2^8}^{1\times 3} 
= \left(
 \begin{matrix}
    1~~0~~0~~0~~0~~0~~0~~0~~1~~1~~1~~1~~1~~1~~1~~1~~1~~1~~0~~0~~0~~0~~0~~1 \\
    0~~1~~0~~0~~0~~0~~0~~0~~1~~1~~0~~0~~0~~1~~1~~1~~1~~1~~0~~1~~1~~0~~0~~0 \\
    0~~0~~1~~0~~0~~0~~0~~0~~1~~1~~0~~1~~1~~0~~1~~1~~0~~1~~1~~0~~1~~1~~0~~0 \\
    0~~0~~0~~1~~0~~0~~0~~0~~1~~1~~0~~1~~0~~1~~0~~1~~0~~0~~1~~1~~0~~1~~1~~0 \\
    0~~0~~0~~0~~1~~0~~0~~0~~1~~1~~0~~1~~0~~0~~1~~0~~0~~0~~0~~1~~1~~0~~1~~1 \\
    0~~0~~0~~0~~0~~1~~0~~0~~0~~1~~1~~0~~1~~0~~0~~1~~1~~0~~1~~1~~0~~1~~0~~1 \\
    0~~0~~0~~0~~0~~0~~1~~0~~1~~0~~0~~0~~1~~1~~0~~0~~1~~1~~1~~0~~0~~0~~1~~0 \\
    0~~0~~0~~0~~0~~0~~0~~1~~0~~1~~0~~0~~0~~1~~1~~0~~0~~1~~1~~1~~0~~0~~0~~1   
  \end{matrix} 
\right) \normalsize\in \mathbb{F}_2^{8\times 24}
$$ 

- $(\alpha_1, \alpha_2,\alpha_3) = (\alpha^0, \alpha^{72}, \alpha^{80})$

$$
\mathbf{H}_{optimal}=\left( \begin{matrix} \alpha^0 & \alpha^{72} & \alpha^{80} \end{matrix} \right) \in \mathbb{F}_{2^8}^{1\times 3} 
= \left(
 \begin{matrix}
    1~~0~~0~~0~~0~~0~~0~~0~~1~~0~~1~~0~~0~~1~~1~~0~~1~~0~~1~~1~~1~~1~~1~~1 \\
    0~~1~~0~~0~~0~~0~~0~~0~~0~~1~~0~~1~~0~~0~~1~~1~~1~~1~~1~~0~~0~~1~~1~~1 \\
    0~~0~~1~~0~~0~~0~~0~~0~~1~~0~~0~~1~~0~~0~~0~~1~~1~~1~~0~~0~~1~~0~~1~~1 \\
    0~~0~~0~~1~~0~~0~~0~~0~~1~~1~~1~~1~~0~~0~~0~~0~~1~~1~~0~~1~~1~~1~~0~~1 \\
    0~~0~~0~~0~~1~~0~~0~~0~~0~~1~~1~~1~~1~~0~~0~~0~~1~~1~~0~~1~~0~~1~~1~~0 \\
    0~~0~~0~~0~~0~~1~~0~~0~~0~~0~~1~~1~~1~~1~~0~~0~~0~~1~~1~~0~~1~~0~~1~~1 \\
    0~~0~~0~~0~~0~~0~~1~~0~~0~~0~~0~~1~~1~~1~~1~~0~~1~~0~~0~~0~~1~~1~~0~~1 \\
    0~~0~~0~~0~~0~~0~~0~~1~~0~~0~~0~~0~~1~~1~~1~~1~~1~~1~~1~~1~~1~~1~~1~~0    
  \end{matrix} 
\right) \normalsize\in \mathbb{F}_2^{8\times 24}
$$ 

- $(\alpha_1, \alpha_2,\alpha_3) = (\alpha^0, \alpha^{8}, \alpha^{183})$

$$
\mathbf{H}_{optimal}=\left( \begin{matrix} \alpha^0 & \alpha^{8} & \alpha^{183} \end{matrix} \right) \in \mathbb{F}_{2^8}^{1\times 3} 
= \left(
 \begin{matrix}
    1~~0~~0~~0~~0~~0~~0~~0~~1~~0~~1~~1~~1~~0~~0~~0~~0~~0~~1~~0~~0~~0~~1~~1 \\
    0~~1~~0~~0~~0~~0~~0~~0~~0~~1~~0~~1~~1~~1~~0~~0~~1~~0~~1~~0~~1~~0~~0~~1 \\
    0~~0~~1~~0~~0~~0~~0~~0~~0~~0~~1~~0~~1~~1~~1~~0~~1~~1~~1~~0~~1~~1~~0~~0 \\
    0~~0~~0~~1~~0~~0~~0~~0~~0~~0~~0~~1~~0~~1~~1~~1~~0~~1~~1~~1~~0~~1~~1~~0 \\
    0~~0~~0~~0~~1~~0~~0~~0~~1~~0~~1~~1~~0~~0~~1~~1~~0~~0~~1~~1~~1~~0~~1~~1 \\
    0~~0~~0~~0~~0~~1~~0~~0~~1~~1~~1~~0~~0~~0~~0~~1~~1~~0~~1~~0~~0~~1~~0~~1 \\
    0~~0~~0~~0~~0~~0~~1~~0~~1~~1~~0~~0~~1~~0~~0~~0~~1~~1~~1~~0~~1~~0~~1~~0 \\
    0~~0~~0~~0~~0~~0~~0~~1~~0~~1~~1~~0~~0~~1~~0~~0~~0~~1~~1~~1~~0~~1~~0~~1   
  \end{matrix} 
\right) \normalsize\in \mathbb{F}_2^{8\times 24}
$$ 